In [12]:
from os import listdir
from os.path import isfile, join
from IPython.display import display
import sys
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
import pandas as pd
mypath = '/home/tommy/myenv/workspace/data/onepunch/dashboard_data/'

In [14]:
import json
from datetime import datetime

def moment_to_df(json_dir):
    taskDict = dict()
    with open(json_dir) as json_file:  
        data = json.load(json_file)
        time = data['created_at']
        timestamp=datetime.strptime(time, '%Y-%m-%dT%H:%M:%S.%fZ')
        taskDict['time'] = timestamp
        all_current_orders = data['orders_data']
        recordNum=len(all_current_orders)
        #print(recordNum)
        #print(all_current_orders[0])
        for i in range(recordNum):
            state = data['orders_data'][i]['state']
            if state in taskDict.keys():
                taskDict[state] += 1
            else:
                taskDict[state] = 1
        taskDf = pd.DataFrame(taskDict, index=[0])
        taskDf["time"] = pd.to_datetime(taskDf["time"], utc=True, errors='coerce')
        taskDf = taskDf.set_index('time')
        return taskDf

def combine_all_json(path):
    all_state_json = [f for f in listdir(mypath) if (isfile(join(mypath, f)) and f.endswith('.json'))]
    list_of_dataframes = []
    total_num = len(all_state_json)
    for i,jsonFile in enumerate(all_state_json):
        try:
            fullDir = join(path,jsonFile)
            one_df = moment_to_df(fullDir)
            list_of_dataframes.append(one_df)
            if i%100 ==0:
                sys.stdout.write(
                    "\r>> total processed: {}%".format(100*i/(total_num+0.0)))
                sys.stdout.flush()
        except Exception as e:
            #print(fullDir,e)
            pass
    print("merging results...")       
    full_df = pd.concat(list_of_dataframes,join='outer',sort=False)
    full_df = full_df.fillna(0)
    full_df = full_df.sort_index(ascending=True)
    print("finished")
    return full_df

#one_df = moment_to_df(dev)
res = combine_all_json(mypath)

>> total processed: 99.8158736312%%merging results...
finished


,extracting_measurements,labeling,markup_verifying,model_segmenting,modeling,pro_measurements_processing,texturing,verifying,waiting_labeling,waiting_markup,...,cv_processing,zulu_processing,waiting_zulu_processing,waiting_pro_measurements_processing,failed_pro_measurements_processing,waiting_primitives_processing,waiting_image_processing,correcting_labels,new,failed_image_processing
time,,,,,,,,,,,,,,,,,,,,,
2018-08-03 21:03:23.998000+00:00,7.0,1.0,0.0,23.0,88.0,8.0,6.0,4.0,52,53,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-08-03 21:04:08.973000+00:00,4.0,1.0,0.0,23.0,86.0,3.0,5.0,5.0,51,53,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-08-03 21:04:35.797000+00:00,2.0,1.0,0.0,22.0,86.0,1.0,6.0,6.0,52,53,...,0.0,0.0,0.0,7.0,0.0,0.0,2.0,0.0,0.0,0.0
2018-08-03 21:05:07.107000+00:00,4.0,2.0,1.0,21.0,85.0,0.0,4.0,6.0,53,53,...,0.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-08-03 21:07:00.874000+00:00,6.0,1.0,1.0,20.0,87.0,1.0,3.0,9.0,53,53,...,0.0,0.0,0.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
from features import *
#file_name = '../data/onepunch/orders_after_2018-06-10_puborders_000'
file_name = '../data/onepunch/puborder_data/orders_after_2018-08-04_puborders_000'

with open('./data_downloader/column.txt') as txtfile:  
    cols = txtfile.read().splitlines()
    
def load_data():
    print("Loading Data... ")
    data= pd.read_csv(file_name,
                      sep='|',
                      index_col=False, 
                      names=cols,
                      low_memory=False
                    )
    print("Data Loaded!")
    return data

raw_data = load_data()

Loading Data... 
Data Loaded!


In [50]:
res.to_pickle('./data/0804-0807.pkl')

In [11]:
res = pd.read_pickle('./data/0804-0807.pkl')
display(res.tail())
res["time"] = pd.to_datetime(res["time"], utc=True, errors='coerce')

,extracting_measurements,labeling,markup_verifying,model_segmenting,modeling,pro_measurements_processing,texturing,verifying,waiting_labeling,waiting_markup,...,cv_processing,zulu_processing,waiting_zulu_processing,waiting_pro_measurements_processing,failed_pro_measurements_processing,waiting_primitives_processing,waiting_image_processing,correcting_labels,new,failed_image_processing
time,,,,,,,,,,,,,,,,,,,,,
2018-08-07 07:23:32.265,2.0,0.0,0.0,9.0,60.0,1.0,2.0,8.0,155,16,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-08-07 07:23:51.538,2.0,0.0,0.0,10.0,60.0,0.0,2.0,7.0,155,14,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-08-07 07:24:13.025,2.0,0.0,0.0,9.0,60.0,0.0,2.0,7.0,155,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-08-07 07:24:35.303,3.0,0.0,0.0,8.0,60.0,0.0,2.0,7.0,155,15,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-08-07 07:24:54.741,4.0,0.0,0.0,8.0,60.0,1.0,1.0,6.0,155,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


KeyError: 'time'

In [4]:
from features import *
#file_name = '../data/onepunch/orders_after_2018-06-10_puborders_000'
file_name = '../data/onepunch/puborder_data/orders_after_2018-08-04_puborders_000'

with open('./data_downloader/column.txt') as txtfile:  
    cols = txtfile.read().splitlines()
    
def load_data():
    print("Loading Data... ")
    data= pd.read_csv(file_name,
                      sep='|',
                      index_col=False, 
                      names=cols,
                      low_memory=False
                    )
    print("Data Loaded!")
    return data

raw_data = load_data()
display(raw_data.head())
#for i in range(139):
#    print(raw_data.iloc[0,i])

Loading Data... 
Data Loaded!


,order_id,order_state,order_created_at,order_updated_at,original_order_id,order_complexity,order_image_score,order_image_score_name,priority,order_deliverable,...,verifying_time_first_completion,texturing_time_total,texturing_time_first_completion,qa_time_total,openings_marking_time_total,openings_marking_time_first_completion,vps_marking_time_total,vps_marking_time_first_completion,primitive_marking_time_total,primitive_marking_time_first_completion
0,849425,complete,2018-08-04 01:29:54.741288,2018-08-06 01:47:48.960142,NaN,Average,3.0,hard_images,8,complete,...,4.639653,0.817808,0.817808,0.327130,0.000000,NaN,0.000000,NaN,0.000000,NaN
1,849542,waiting_qa,2018-08-04 08:11:34.202532,2018-08-04 10:01:49.499548,845361.0,Average,1.0,easy_images,10,complete,...,0.055874,0.894428,0.894428,0.000000,0.000000,NaN,0.000000,NaN,0.000000,NaN
2,850041,waiting_qa,2018-08-04 14:22:00.739828,2018-08-06 08:38:56.079382,844261.0,Average,1.0,easy_images,10,complete,...,0.387858,0.627368,0.627368,0.000000,0.000000,NaN,0.000000,NaN,0.000000,NaN
3,850565,complete,2018-08-04 20:56:05.594195,2018-08-06 01:41:35.204171,NaN,Simple,1.0,easy_images,8,complete,...,2.160056,0.584444,0.584444,0.366276,3.257213,3.257213,1.978986,1.978986,3.760403,3.760403
4,850633,waiting_qa,2018-08-04 21:57:34.583366,2018-08-05 01:06:12.625672,834926.0,Average,2.0,medium_images,10,roof,...,0.370352,0.192518,0.192518,0.000000,0.000000,NaN,0.000000,NaN,0.000000,NaN


# end

In [36]:
all_state_json = [f for f in listdir(mypath) if (isfile(join(mypath, f)) and f.endswith('.json'))]
dev = join(mypath,all_state_json[1900])
one_df = moment_to_df(dev)
display(one_df)

columns = len(list(one_df))
total = 0
for i in range(columns):
    total += one_df.iloc[0][i]
print(total)

542


In [43]:
sorted_res = res.sort_index(ascending=True)
display(sorted_res.iloc[8900:8910])

,extracting_measurements,labeling,markup_verifying,model_segmenting,modeling,pro_measurements_processing,texturing,verifying,waiting_labeling,waiting_markup,...,cv_processing,zulu_processing,waiting_zulu_processing,waiting_pro_measurements_processing,failed_pro_measurements_processing,waiting_primitives_processing,waiting_image_processing,correcting_labels,new,failed_image_processing
time,,,,,,,,,,,,,,,,,,,,,
2018-08-06 19:18:21.251,0.0,1.0,2.0,10.0,42.0,1.0,2.0,5.0,134,27,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-08-06 19:18:43.730,0.0,1.0,2.0,11.0,42.0,0.0,2.0,5.0,133,25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-08-06 19:19:09.204,2.0,0.0,2.0,11.0,41.0,0.0,2.0,4.0,133,24,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-08-06 19:19:30.080,2.0,2.0,2.0,11.0,41.0,1.0,2.0,5.0,132,24,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-08-06 19:19:53.367,3.0,3.0,2.0,11.0,40.0,2.0,1.0,5.0,130,23,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-08-06 19:20:26.292,2.0,3.0,2.0,11.0,40.0,2.0,1.0,5.0,132,22,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-08-06 19:20:47.471,3.0,3.0,2.0,10.0,38.0,1.0,2.0,4.0,132,22,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-08-06 19:21:06.946,2.0,3.0,1.0,9.0,38.0,2.0,1.0,5.0,131,21,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-08-06 19:21:31.903,2.0,4.0,3.0,8.0,38.0,1.0,1.0,5.0,133,23,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
d1 = pd.DataFrame({'one' : ['2018-08-07 07:20:32.265000+00:00',
                           '2018-08-07 08:26:32.265000+00:00'], 
                           'two' : [1,2],
                           'three':[123,456]})
d2 = pd.DataFrame({'x' : ['2018-08-07 07:26:32.265000+00:00',
                         '2018-08-07 08:26:32.265000+00:00'], 'y' : [100,200]})
d1["one"] = pd.to_datetime(d1["one"], utc=True, errors='coerce')
d1 = d1.set_index('one')
d2["x"] = pd.to_datetime(d2["x"], utc=True, errors='coerce')
d2 = d2.set_index('x')
d1.sort_index(ascending=True, inplace=True)
d2.sort_index(ascending=True, inplace=True)
print('d1',d1)
print('d2',d2)
print(d1.shape)
d1 = d1.reindex(d2.index, method='nearest',tolerance = '1min')
d1.dropna(inplace=True)
print(d1.shape)
d2 = d2.join(d1, how='right')
print(d2)

('d1',                                   three  two
one                                         
2018-08-07 07:20:32.265000+00:00    123    1
2018-08-07 08:26:32.265000+00:00    456    2)
('d2',                                     y
x                                    
2018-08-07 07:26:32.265000+00:00  100
2018-08-07 08:26:32.265000+00:00  200)
(2, 2)
(1, 2)
                                    y  three  two
x                                                
2018-08-07 08:26:32.265000+00:00  200  456.0  2.0


['order_id', 'order_state', 'order_created_at', 'order_updated_at', 'original_order_id', 'order_complexity', 'order_image_score', 'order_image_score_name', 'priority', 'order_deliverable', 'order_most_recent_failure_reason', 'site_id', 'order_local', 'slayer_job_id', 'order_site_name', 'sneaky', 'order_skip_markups', 'number_of_stories', 'e_number_of_stories_name', 'complexity_attributes_total_roof_facets_above_4_sqft', 'complexity_attributes_total_roof_facets', 'complexity_attributes_roof_square_footage', 'complexity_attributes_walls_square_footage', 'complexity_attributes_is_mfr', 'map_attributes_source', 'slayer_attributes_allows_mfr', 'slayer_attributes_is_mfr', 'slayer_attributes_user_first_capture', 'poser_release_number', 'structure_type', 'structure_type_name', 'poser_solve_status', 'hover_now', 'foundational_plus', 'first_qa_datetime', 'first_qa_date', 'models_reprocessed', 'order_needs_to_be_reprocessed_flag', 'special_request', 'original_roof', 'first_qa_for_state', 'last_qa